1. Overview
2. EDA
3. Model Details
4. Inference



### Overview

In this competition, we introduce a dataset of 21,367 labeled images collected during a regular survey in Uganda. Most images were crowdsourced from farmers taking photos of their gardens, and annotated by experts at the National Crops Resources Research Institute (NaCRRI) in collaboration with the AI lab at Makerere University, Kampala. This is in a format that most realistically represents what farmers would need to diagnose in real life.

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torchvision
from PIL import Image
import albumentations
import json
from tqdm import tqdm
# import cv2
from sklearn import metrics , model_selection, preprocessing

%matplotlib inline 


In [ ]:
BASE_DIR = '../input/cassava-leaf-disease-classification'
df = pd.read_csv(os.path.join(BASE_DIR,'train.csv'))
# df = df[:100] 
df['path'] = df['image_id'].map(lambda x: os.path.join(BASE_DIR,'train_images',x))

In [ ]:
df.drop(columns=['image_id'],inplace=True)
df = df.reset_index(drop=True)
df.head(10)

### EDA
We have the five categories to predict and number of records are highly biased  

In [ ]:
df.label.value_counts()

In [ ]:
df.plot.hist(df.label,figsize=(10,5))

We have 5 different types of category 
* Cassava Bacterial Blight (CBB)
* Cassava Brown Streak Disease (CBSD)
* Cassava Green Mottle (CGM)
* Cassava Mosaic Disease (CMD)
* Healthy


In [ ]:
with open('../input/cassava-leaf-disease-classification/label_num_to_disease_map.json') as fn:
    print(json.loads(fn.read()))
    

In [ ]:
print(len(df))

In [ ]:
img = Image.open(df['path'][1])
w,b = img.size
print(w,b)

In [ ]:
df.head()
df_train, df_valid = model_selection.train_test_split( df, test_size =0.15, random_state = 42, stratify=df.label.values)
df_train = df_train.reset_index(drop=True)
df_valid = df_valid.reset_index(drop=True)

In [ ]:
df_valid.head()

In [ ]:
BATCH_SIZE=64
EPOCHS=10
VALID_BATCH_SIZE=128
MODEL_PATH = '/kaggle/working'


#### Show sample images

In [ ]:
import random 
def show_image_using_path(image,label):
    img =Image.open(image)
#     plt.figure(figsize=(10,10))
    plt.imshow(img)
    lbl =label 
    plt.title(f'Class: {lbl}')
    plt.axis('off')

plt.figure(figsize=(16, 12))
for i in range(5):
    df_temp = df_train[df_train["label"] == i]
    plt.subplot(3,3,i+1)
    ranom_number = random.randint(0,100)
    show_image_using_path(list(df_temp['path'])[ranom_number], list(df_temp['label'])[ranom_number] )



#### Create the dataset for images

In [ ]:
class LeafDataset:
    def __init__(self, img_path, labels, resize, albumentations=None):
        self.img_path = img_path
        self.labels = labels
        self.resize = resize
        self.albumentations =albumentations
    
    def __len__(self):
        return len(self.img_path)
    
    def __getitem__(self, item):
        labels = self.labels[item]
        img = Image.open(self.img_path[item])
        img = img.resize((self.resize[0],self.resize[1]),resample=Image.BILINEAR)
        img = np.array(img)
        if self.albumentations is not None:
            augmented_imgs = self.albumentations(image=img)
            img = augmented_imgs["image"]

            img= np.transpose(img, (2,0,1)).astype(np.float32)
        return {
            'image': torch.tensor(img, dtype=torch.float),
            'label' : torch.tensor(labels,dtype=torch.long)
        }

#### Test dataset 

In [ ]:
def show_image_tensor(image_tensor,label):
    print(image_tensor.shape)
    plt.figure(figsize=(10,10))
    plt.title(f'Class: {label}')
    plt.imshow(img)
    plt.axis('off')

#### Augmnet images with albumentation library

In [ ]:
train_albumentations = albumentations.Compose([
            albumentations.RandomResizedCrop(400,300),
#             albumentations.Transpose(p=0.5),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.ShiftScaleRotate(p=0.5),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            )], p=1.)

valid_albumentations = albumentations.Compose([
             albumentations.RandomResizedCrop(400,300),
            albumentations.Resize(400,300),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            )], p=1.)



In [ ]:
train_dataset= LeafDataset(df_train.path.values,df_train.label.values,(400,300),train_albumentations)
temp_dataset= train_dataset[0] 
show_image_tensor(temp_dataset['image'], temp_dataset['label'])

In [ ]:

valid_dataset= LeafDataset(df_valid.path.values,df_valid.label.values,(400,300),valid_albumentations)
temp_dataset= valid_dataset[0] 
show_image_tensor(temp_dataset['image'], temp_dataset['label'])

#### Create dataloader

In [ ]:
train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE,num_workers=2)
valid_data_loader = torch.utils.data.DataLoader(valid_dataset,batch_size=VALID_BATCH_SIZE,num_workers=2)

#### Create model : We used resnet 18

In [ ]:
class CassavaModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.resnet= torchvision.models.resnet18(pretrained=True)
        self.resnet.fc = nn.Linear(512,num_classes)
        self.step_scheduler_after = 'epoch'
#         print(self.resnet)
    
    
    def forward(self,image,labels=None):
        batch_size , _, _,_ = image.shape
        outputs = self.resnet(image)        
        return outputs
    

#### Test the model 

In [ ]:
cm = CassavaModel(5)
print(cm.resnet)
img = torch.rand((1, 3, 50, 200))
x = cm(img, torch.rand((1, 5)))

define utilities function

In [ ]:
def loss_fn(outputs,labels):
    return  nn.CrossEntropyLoss()(outputs,labels)

def train_fn(data_loader, model, optimizer, device, scheduler):
    model.train()
    
    for bi , data in tqdm(enumerate(data_loader),total=len(data_loader)):
        labels = data['label']
        images = data['image']
        
        labels = labels.to(device,dtype=torch.long)
        images = images.to(device,dtype=torch.float)
        
        optimizer.zero_grad()
        outputs = model(images,labels)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
def eval_fn(data_loader, model, device):
    model.eval()
    final_targets = []
    final_outputs = []
    
    with torch.no_grad():
        for bi,data in tqdm(enumerate(data_loader),total=len(data_loader)):
            labels = data['label']
            images = data['image']
            
            labels= labels.to(device, dtype=torch.long)
            images = images.to(device,dtype=torch.float)
            
            outputs = model(images,labels)
            final_targets.extend(labels.tolist())
            outputs = torch.argmax(outputs, dim=1)
#             print(outputs)
            final_outputs.extend(outputs.cpu())
            accuracy = metrics.accuracy_score(labels.cpu(), outputs.cpu())
    return accuracy,final_outputs

     
def pred_fn(data_loader, model, device):
    model.eval()
    final_targets = []
    final_outputs = []
    
    with torch.no_grad():
        for bi,data in tqdm(enumerate(data_loader),total=len(data_loader)):
            labels = data['label']
            images = data['image']
            
            labels= labels.to(device, dtype=torch.long)
            images = images.to(device,dtype=torch.float)
            
            outputs = model(images,labels)
            final_targets.extend(labels.tolist())
            outputs = torch.argmax(outputs, dim=1)
            final_outputs.extend(outputs.cpu())
           
    return final_outputs
    

See the model structure

In [ ]:
device = torch.device('cuda')
model = CassavaModel(num_classes=5)
model.to(device)

In [ ]:
# image =train_dataset[0]['image'].unsqueeze(0)
# label =train_dataset[0]['label'].unsqueeze(0)
# model(image,label)

In [ ]:

num_train_steps = int(len(train_dataset)/BATCH_SIZE *EPOCHS)
optimizer = torch.optim.Adam(model.parameters(),lr=3e-4)
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_train_steps)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.8, patience=5, verbose=True)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=0.7)

best_accuracy =0 
for epoch in range(EPOCHS):
    train_fn(train_data_loader, model,optimizer,device,scheduler)
    accuracy,final_preds = eval_fn(valid_data_loader,model,device)
    if accuracy >= best_accuracy:
#         torch.save(model.state_dict(), MODEL_PATH)
        best_accuracy = accuracy
        print(best_accuracy)


In [ ]:
test_df = pd.read_csv("../input/cassava-leaf-disease-classification/sample_submission.csv")
test_df['path'] = test_df['image_id'].map(lambda x: os.path.join(BASE_DIR,'test_images',x))
# fake targets
# test_df.drop(columns=['image_id'],inplace=True)
test_targets = test_df.label.values
test_df.reset_index(drop=True)

test_df.head(10)

In [ ]:
test_albumentations = albumentations.Compose([
             albumentations.RandomResizedCrop(400,300),
            albumentations.Resize(400,300),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            )], p=1.)



test_dataset = LeafDataset(test_df.path.values,test_df.label.values,(256,256),test_albumentations)
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE,num_workers=2)

In [ ]:
final_preds = pred_fn(test_data_loader,model,device)
test_df.drop(columns=['path'],inplace=True)
test_df.label = int(final_preds[0])
test_df.head()
test_df.to_csv("submission.csv", index=False)

In [ ]:
print('Done')